In [31]:
#!/usr/bin/env python
import os
import sys
import rospy
import copy
import tf
import cv2
from cv_bridge import CvBridge, CvBridgeError
import numpy as np
import std_msgs.msg
from std_msgs.msg import UInt16, String
import geometry_msgs.msg
from math import *
from numpy.linalg import inv
import scipy.io
from mat4py import loadmat
import actionlib
from robotiq_2f_gripper_msgs.msg import CommandRobotiqGripperFeedback, CommandRobotiqGripperResult, CommandRobotiqGripperAction, CommandRobotiqGripperGoal
from robotiq_2f_gripper_control.robotiq_2f_gripper_driver import Robotiq2FingerGripperDriver as Robotiq
from sensor_msgs.msg import Image, CameraInfo
import message_filters
from visualization_msgs.msg import Marker
from visualization_msgs.msg import MarkerArray
from geometry_msgs.msg import Point
import random
import urx
import math3d as m3d
import logging
import matplotlib.pyplot as plt
rospy.init_node('experiment_single_object', anonymous=True) #initialize the node



In [32]:
rospy.logwarn("Client test: Starting sending goals")
rob = urx.Robot("192.168.1.102")
logging.basicConfig(level=logging.WARN)
rob.set_tcp((0, 0, 0, 0, 0, 0))

[WARN] [1612537193.359949]: Client test: Starting sending goals


In [33]:
action_name = rospy.get_param('~action_name', 'command_robotiq_action')
robotiq_client = actionlib.SimpleActionClient(action_name, CommandRobotiqGripperAction)
robotiq_client.wait_for_server()

True

In [34]:
import serial
import signal
import time

serialPort = "/dev/Arduino"
baudRate = 57600
ser = serial.Serial(serialPort, baudRate, timeout=0.5)
def myHandler(signum, frame):
    pass
def softtip():
    signal.signal(signal.SIGALRM, myHandler)
    signal.setitimer(signal.ITIMER_REAL, 0.01)
    ser.write('2000')
    signal.setitimer(signal.ITIMER_REAL, 0)
    time.sleep(1.5)
    signal.setitimer(signal.ITIMER_REAL, 0.01)
    ser.write('2090')
    signal.setitimer(signal.ITIMER_REAL, 0)
    time.sleep(0.1)

    
def hardtip():
    signal.signal(signal.SIGALRM, myHandler)
    signal.setitimer(signal.ITIMER_REAL, 0.01)
    ser.write('2180')
    signal.setitimer(signal.ITIMER_REAL, 0)
    time.sleep(1.5)
    signal.setitimer(signal.ITIMER_REAL, 0.01)
    ser.write('2090')
    signal.setitimer(signal.ITIMER_REAL, 0)
    time.sleep(0.1)

In [35]:
def go_to_home():
    
    Hong_joint0 = radians(101.23)
    Hong_joint1 = radians(-83.77)
    Hong_joint2 = radians(-130.28)
    Hong_joint3 = radians(-55.97)
    Hong_joint4 = radians(89.33)
    Hong_joint5 = radians(-78.73)
    
    rob.movej((Hong_joint0,Hong_joint1, Hong_joint2, Hong_joint3, Hong_joint4, Hong_joint5), 0.6, 1.5)
    
def go_to_safe_place():
    
    Hong_joint0 = radians(101.70)
    Hong_joint1 = radians(-75.56)
    Hong_joint2 = radians(-116.96)
    Hong_joint3 = radians(-77.49)
    Hong_joint4 = radians(89.42)
    Hong_joint5 = radians(-78.13)
    
    rob.movej((Hong_joint0,Hong_joint1, Hong_joint2, Hong_joint3, Hong_joint4, Hong_joint5), 0.6, 1.5)
    
def go_to_end_place():
    
    Hong_joint0 = radians(106.13)
    Hong_joint1 = radians(-60.48)
    Hong_joint2 = radians(-130.69)
    Hong_joint3 = radians(-78.86)
    Hong_joint4 = radians(89.68)
    Hong_joint5 = radians(-73.78)
    
    rob.movej((Hong_joint0,Hong_joint1, Hong_joint2, Hong_joint3, Hong_joint4, Hong_joint5), 0.6, 1.5)

In [36]:
image_color_sub = message_filters.Subscriber('/camera/color/image_raw', Image)
image_aligned_depth_sub = message_filters.Subscriber('/camera/aligned_depth_to_color/image_raw', Image)
ts = message_filters.TimeSynchronizer([image_color_sub, image_aligned_depth_sub], 10)
ts.registerCallback(image_callback)

NameError: name 'image_callback' is not defined

In [37]:
def finger_tip_position_wrt_gripper_frame(alpha, l0, l1, l2l, l2r, flength_l, flength_r):
    alpha=alpha*pi/180
    left_fingertip_position=[-l0-l1*sin(alpha)+l2l, -l1*cos(alpha)-flength_l]
    right_fingertip_position=[l0+l1*sin(alpha)-l2r, -l1*cos(alpha)-flength_r]
    return left_fingertip_position, right_fingertip_position

def pos_flength_adjust_finger_fixed(current_alpha, next_alpha, theta, l0, l1, l2l, l2r, flength_l):
    theta=theta*pi/180
    current_flength_r = flength_l + (2*l0+2*l1*sin(current_alpha*pi/180)-l2l-l2r)/tan(theta)
    current_left_fingertip_pos_g, current_right_fingertip_pos_g = finger_tip_position_wrt_gripper_frame(current_alpha, l0, l1, l2l, l2r, flength_l, current_flength_r)
    transforming_matrix_w_g = np.array([[sin(theta), -cos(theta), -current_left_fingertip_pos_g[0]*sin(theta)+current_left_fingertip_pos_g[1]*cos(theta)], [cos(theta), sin(theta), -current_left_fingertip_pos_g[0]*cos(theta)-current_left_fingertip_pos_g[1]*sin(theta)], [0, 0, 1]])
    current_right_fingertip_pos_w = (np.squeeze(np.dot(transforming_matrix_w_g, np.array([[current_right_fingertip_pos_g[0]], [current_right_fingertip_pos_g[1]], [1]])))[0:2]).tolist()
    next_left_fingertip_pos_g, next_right_fingertip_pos_g = finger_tip_position_wrt_gripper_frame(next_alpha, l0, l1, l2l, l2r, flength_l, current_flength_r)
    next_left_fingertip_pos_w = (np.squeeze(np.dot(transforming_matrix_w_g, np.array([[next_left_fingertip_pos_g[0]], [next_left_fingertip_pos_g[1]], [1]])))[0:2]).tolist()
    next_right_fingertip_pos_w = (np.squeeze(np.dot(transforming_matrix_w_g, np.array([[next_right_fingertip_pos_g[0]], [next_right_fingertip_pos_g[1]], [1]])))[0:2]).tolist()
    translation_dir_dis = [-next_left_fingertip_pos_w[0], -next_left_fingertip_pos_w[1]]
    next_right_fingertip_pos_after_trans_w = [next_right_fingertip_pos_w[0]+translation_dir_dis[0], next_right_fingertip_pos_w[1]+translation_dir_dis[1]]
    right_finger_shorten_distance=abs(next_right_fingertip_pos_after_trans_w[1])/sin(theta)
    translation_dir_dis[0] = translation_dir_dis[0]+(2*l0+2*l1*sin(current_alpha*pi/180)-l2l-l2r)/sin(theta)-(next_right_fingertip_pos_after_trans_w[0]-right_finger_shorten_distance*cos(theta))
    return translation_dir_dis, right_finger_shorten_distance

def pos_flength_adjust_thumb_fixed(current_alpha, next_alpha, theta, l0, l1, l2l, l2r, flength_l):
    theta=theta*pi/180
    current_flength_r = flength_l + (2*l0+2*l1*sin(current_alpha*pi/180)-l2l-l2r)/tan(theta)
    current_left_fingertip_pos_g, current_right_fingertip_pos_g = finger_tip_position_wrt_gripper_frame(current_alpha, l0, l1, l2l, l2r, flength_l, current_flength_r)
    transforming_matrix_w_g = np.array([[sin(theta), -cos(theta), -current_left_fingertip_pos_g[0]*sin(theta)+current_left_fingertip_pos_g[1]*cos(theta)], [cos(theta), sin(theta), -current_left_fingertip_pos_g[0]*cos(theta)-current_left_fingertip_pos_g[1]*sin(theta)], [0, 0, 1]])
    current_right_fingertip_pos_w = (np.squeeze(np.dot(transforming_matrix_w_g, np.array([[current_right_fingertip_pos_g[0]], [current_right_fingertip_pos_g[1]], [1]])))[0:2]).tolist()
    next_left_fingertip_pos_g, next_right_fingertip_pos_g = finger_tip_position_wrt_gripper_frame(next_alpha, l0, l1, l2l, l2r, flength_l, current_flength_r)
    next_left_fingertip_pos_w = (np.squeeze(np.dot(transforming_matrix_w_g, np.array([[next_left_fingertip_pos_g[0]], [next_left_fingertip_pos_g[1]], [1]])))[0:2]).tolist()
    next_right_fingertip_pos_w = (np.squeeze(np.dot(transforming_matrix_w_g, np.array([[next_right_fingertip_pos_g[0]], [next_right_fingertip_pos_g[1]], [1]])))[0:2]).tolist()
    translation_dir_dis = [-next_left_fingertip_pos_w[0], -next_left_fingertip_pos_w[1]]
    next_right_fingertip_pos_after_trans_w = [next_right_fingertip_pos_w[0]+translation_dir_dis[0], next_right_fingertip_pos_w[1]+translation_dir_dis[1]]
    right_finger_shorten_distance=abs(next_right_fingertip_pos_after_trans_w[1])/sin(theta)
    return translation_dir_dis, right_finger_shorten_distance

In [49]:
from geometry_msgs.msg import WrenchStamped
from std_msgs.msg import UInt16
import time

def skim_grasping():
    go_to_safe_place()
    Robotiq.goto(robotiq_client, pos=13*0.14/48.0+1.12/48.0, speed=0.5, force=10, block=True)  
    signal.signal(signal.SIGALRM, myHandler)
    signal.setitimer(signal.ITIMER_REAL, 0.005)
    ser.write('1180')
    signal.setitimer(signal.ITIMER_REAL, 0)
    time.sleep(0.2)    
    softtip()
    rob.set_tcp((0.0085, 0, 0.372, 0, 0, 0))  
    rospy.sleep(0.5)
    world2tcp = rob.get_pose()
    world2tcp.pos.x+=0.007
    world2tcp.pos.y-=0.001
    rob.translate((0.05-world2tcp.pos.x,0.675-world2tcp.pos.y,0), acc=0.5, vel=0.8, wait=True) 
    rob.translate((0, 0, -0.095), acc=0.5, vel=0.8) #0.116  #-0.104
    rob.movel_tool((0,0,0,0,0,-pi/2), acc=0.5, vel=0.8, wait=True)
    rob.translate_tool((-0.01, 0, 0), acc=0.06, vel=0.08)
    rob.movel_tool((0,0,0,0,45*pi/180,0), acc=0.3, vel=0.8, wait=True)    
    rob.translate((0, 0, -0.02), acc=0.01, vel=0.05, wait=False)
    wrench = rospy.wait_for_message('/robotiq_ft_wrench', WrenchStamped, timeout = None)
    ini_wrench = wrench.wrench.torque.y
    time0 = time.time()
    num_large_force = 0
    while True:
        if num_large_force == 3:
            rob.stopl()
            break
        wrench = rospy.wait_for_message('/robotiq_ft_wrench', WrenchStamped, timeout = None)
        if wrench.wrench.torque.y>ini_wrench+0.1:   
            num_large_force += 1
        if time.time()-time0>2.8:  
            break
    time.sleep(0.5)
    
    wrench = rospy.wait_for_message('/robotiq_ft_wrench', WrenchStamped, timeout = None)
    ini_wrench = wrench.wrench.torque.y
    rob.translate_tool((0, 0, 0.005), acc=0.01, vel=0.05, wait=False)
    for k0 in range(0, -31, -2):
        signal.setitimer(signal.ITIMER_REAL, 0.001)
        ser.write('1'+str(180+k0))
        signal.setitimer(signal.ITIMER_REAL, 0)
        time.sleep(0.1)
        current_fingerlength_int=180+k0
        wrench = rospy.wait_for_message('/robotiq_ft_wrench', WrenchStamped, timeout = None)
        if wrench.wrench.torque.y>ini_wrench+0.3:
            break
    angle_step=0.2
    for angle in np.arange(13,-2,-angle_step):
        translate_dir_dis, shorten_distance = pos_flength_adjust_finger_fixed(angle, angle-angle_step, 60, 0.0125, 0.1, 0.006, 0.019, 0.072)
        old_fingerlength_int=current_fingerlength_int
        current_fingerlength_int=round(max(current_fingerlength_int-shorten_distance*(180.0/0.03),0))
        signal.signal(signal.SIGALRM, myHandler)
        signal.setitimer(signal.ITIMER_REAL, 0.005)
        ser.write('1'+str(current_fingerlength_int))
        signal.setitimer(signal.ITIMER_REAL, 0)   
        if abs(current_fingerlength_int-old_fingerlength_int)!=0:
            Robotiq.goto(robotiq_client, pos=(angle-1)*0.14/48.0+1.12/48.0, speed=0.2, force=10, block=False)
            rob.translate_tool((sin(60*pi/180)*translate_dir_dis[0]+cos(60*pi/180)*translate_dir_dis[1], 0, cos(60*pi/180)*translate_dir_dis[0]-sin(60*pi/180)*translate_dir_dis[1]), acc=0.1, vel=0.4, wait=True)
        else:
            time.sleep(0.01)
    Robotiq.goto(robotiq_client, pos=-2*0.14/48.0+1.12/48.0, speed=0.2, force=10, block=False)
    rospy.sleep(0.5)
    rob.translate_tool((0, 0, -0.12), acc=0.3, vel=0.8, wait=True)
    #go_to_end_place()
    Robotiq.goto(robotiq_client, pos=3*0.14/48.0+1.12/48.0, speed=0.5, force=10, block=True)  

In [45]:
def scoop():
    go_to_safe_place()
    signal.setitimer(signal.ITIMER_REAL, 0.001)
    ser.write('1000')
    signal.setitimer(signal.ITIMER_REAL, 0)
    time.sleep(1)
    Robotiq.goto(robotiq_client, pos=13*0.14/48.0+1.12/48.0, speed=0.2, force=10, block=False)
    softtip()
    Hong_joint0 = radians(81.54)
    Hong_joint1 = radians(-94.03)
    Hong_joint2 = radians(-105.84)
    Hong_joint3 = radians(-74.76)
    Hong_joint4 = radians(118.75)
    Hong_joint5 = radians(-189.52)
    rob.movej((Hong_joint0,Hong_joint1, Hong_joint2, Hong_joint3, Hong_joint4, Hong_joint5), 0.6, 1.5)
    rob.translate((0, 0, -0.09), acc=0.3, vel=0.8, wait=True)
    rob.translate((0, 0, -0.03), acc=0.01, vel=0.05, wait=False)   
                
    wrench = rospy.wait_for_message('/robotiq_ft_wrench', WrenchStamped, timeout = None)
    ini_wrench = wrench.wrench.torque.y
    time0 = time.time()
    num_large_force = 0
    while True:
        if num_large_force == 5:
            rob.stopl()
            break
        wrench = rospy.wait_for_message('/robotiq_ft_wrench', WrenchStamped, timeout = None)
        if wrench.wrench.torque.y>ini_wrench+0.15:
            num_large_force += 1
        if time.time()-time0>2.8: # need change
            break

    wrench = rospy.wait_for_message('/robotiq_ft_wrench', WrenchStamped, timeout = None)
    ini_wrench = wrench.wrench.torque.y
    for current_finger_length in np.arange(0,184,4):
        signal.setitimer(signal.ITIMER_REAL, 0.001)
        ser.write('1'+str(current_finger_length))
        signal.setitimer(signal.ITIMER_REAL, 0)
        time.sleep(0.2)        
        current_fingerlength_int=current_finger_length
        wrench = rospy.wait_for_message('/robotiq_ft_wrench', WrenchStamped, timeout = None)
        if wrench.wrench.torque.y>ini_wrench+0.3:
            break
    angle_step=0.2
    for angle in np.arange(13,-2,-angle_step):
        translate_dir_dis, shorten_distance = pos_flength_adjust_thumb_fixed(angle, angle-angle_step, 60, 0.0125, 0.1, 0.006, 0.019, 0.072)
        old_fingerlength_int=current_fingerlength_int
        current_fingerlength_int=round(max(current_fingerlength_int-shorten_distance*(160.0/0.025),0))
        signal.signal(signal.SIGALRM, myHandler)
        signal.setitimer(signal.ITIMER_REAL, 0.005)
        ser.write('1'+str(current_fingerlength_int))
        signal.setitimer(signal.ITIMER_REAL, 0)   
        #if abs(current_fingerlength_int-old_fingerlength_int)!=0:
        if True:
            Robotiq.goto(robotiq_client, pos=(angle-1)*0.14/48.0+1.12/48.0, speed=0.2, force=10, block=False)
            rob.translate_tool((sin(60*pi/180)*translate_dir_dis[0]+cos(60*pi/180)*translate_dir_dis[1], 0, cos(60*pi/180)*translate_dir_dis[0]-sin(60*pi/180)*translate_dir_dis[1]), acc=0.1, vel=0.4, wait=True)
        #else:
            #time.sleep(0.01)
    Robotiq.goto(robotiq_client, pos=-2*0.14/48.0+1.12/48.0, speed=0.2, force=10, block=False)
    rospy.sleep(0.5)
    rob.translate_tool((0, 0, -0.12), acc=0.3, vel=0.8, wait=True)
    #go_to_safe_place()
    Robotiq.goto(robotiq_client, pos=3*0.14/48.0+1.12/48.0, speed=0.5, force=10, block=True)  

In [12]:
def find_alpha_from_distance(distance, l0, l1, l2l, l2r):
    alpha = asin((distance+l2l+l2r-2*l0)/2/l1)*180/pi
    return alpha

def direct_pick_execute(Go_index):
    global ver_pose_Go_stone_set, gripper_direction_set, gripper_aperture_set
    go_to_safe_place()
    hardtip()
    Robotiq.goto(robotiq_client, pos=0.75*gripper_aperture_set[Go_index]+0.0105, speed=0.1, force=10, block=True)
    signal.signal(signal.SIGALRM, myHandler)
    signal.setitimer(signal.ITIMER_REAL, 0.005)
    ser.write('1000')
    signal.setitimer(signal.ITIMER_REAL, 0)
    rob.set_tcp((0, 0, 0, 0, 0, 0))
    rospy.sleep(0.5)
    pos_x = rob.get_pose().pos.x
    pos_y = rob.get_pose().pos.y
    pos_z = rob.get_pose().pos.z
    rospy.sleep(0.5)
    rob.translate((ver_pose_Go_stone_set[Go_index][0]-pos_x-0.005,ver_pose_Go_stone_set[Go_index][1]-pos_y+0.0015,0), acc=0.8, vel=2, wait=True)
    rob.movel_tool((0,0,0,0,0,gripper_direction_set[Go_index]),acc=0.8, vel=2)
    rob.translate_tool((0.005,0,0),acc=0.1, vel=0.2)
    #rob.translate((0,0,-0.025), acc=0.1, vel=0.4, wait=True)
    rob.translate((0,0,max((ver_pose_Go_stone_set[Go_index][2]-(pos_z-0.32210))*0.7,-0.035)), acc=0.1, vel=0.1, wait=True)
    rob.translate((0,0,max((ver_pose_Go_stone_set[Go_index][2]-(pos_z-0.32210))*0.6,-0.035)), acc=0.001, vel=0.01, wait=True)
    Robotiq.goto(robotiq_client, pos=-7*0.14/48.0+1.12/48.0, speed=0.1, force=0.3, block=False)
    rospy.sleep(1)
    rob.translate((0, 0, 0.08), acc=0.3, vel=0.8)
    go_to_end_place()
    rob.translate((0,0,-0.07), acc=0.3, vel=0.8, wait=True)
    Robotiq.goto(robotiq_client, pos=0.019, speed=0.5, force=10, block=True)
    softtip()
    rob.translate((0,0,0.07), acc=0.3, vel=0.8, wait=True)

In [13]:
from shapely.affinity import rotate
from shapely.affinity import translate
from shapely.geometry import Point
from shapely.geometry import LinearRing

def direct_pick_collision_check(Go_index):
    global ver_pose_Go_stone_set, gripper_direction_set, gripper_aperture_set
    finger_profile = [[0.01, gripper_aperture_set[Go_index]/2+0.019], [-0.01, gripper_aperture_set[Go_index]/2+0.019], [-0.01, -gripper_aperture_set[Go_index]/2-0.007], [0.01, -gripper_aperture_set[Go_index]/2-0.007]]
    finger_profile = rotate(LinearRing(finger_profile), -gripper_direction_set[Go_index], origin=Point([0,0]), use_radians=True)
    finger_profile = translate(LinearRing(finger_profile), xoff=ver_pose_Go_stone_set[Go_index][0], yoff=ver_pose_Go_stone_set[Go_index][1], zoff=0.0)
    if finger_profile.within(Point([0.05,0.69]).buffer(0.055))==True:
        return False
    else:
        return True

In [14]:
def distane_to_bowl_center(point_coordinate):
    return sqrt((point_coordinate[0]-0.05)**2+(point_coordinate[1]-0.69)**2)

stone_img_count = 0
ver_json_error = False
hori_json_error = False
if 'switching_stone_coordinate_set' in dir():
    while True:
        whether_inherent = raw_input("Whether inherent switching_stone_coordinate_set before? (y or n)")
        if whether_inherent == 'n':
            switching_stone_coordinate_set=[]
            break
        elif whether_inherent == 'y':
            break
else:
    switching_stone_coordinate_set = []
while True:
    while True:
        if_continue = raw_input("continue?")
        if if_continue != '':
            break
    go_to_safe_place()
    signal.signal(signal.SIGALRM, myHandler)
    signal.setitimer(signal.ITIMER_REAL, 0.005)
    ser.write('1000')
    signal.setitimer(signal.ITIMER_REAL, 0)    
    Robotiq.goto(robotiq_client, pos=0.023, speed=0.5, force=10, block=True)
    rospy.sleep(1)
    go_to_home()
    rospy.sleep(1)
    hori_Go_stone_number = None
    ver_Go_stone_number = None
    Go_stone_dir_sorted = []
    hori_pose_Go_stone_set = []
    yaw_pitch_normal_set = []
    hori_box_set = []
    ver_pose_Go_stone_set = []
    gripper_direction_set = []
    gripper_aperture_set = []
    is_send_stone_img = 1
    print 'started', stone_img_count
    while True:
        if hori_Go_stone_number != None and hori_Go_stone_number == len(hori_box_set) and ver_Go_stone_number != None and ver_Go_stone_number == len(gripper_aperture_set):
            break
        elif hori_Go_stone_number != None and hori_Go_stone_number == len(hori_box_set) and ver_Go_stone_number != None and ver_json_error==True:
            rospy.sleep(0.5)
            break   
        elif hori_Go_stone_number != None and hori_json_error==True:
            rospy.sleep(0.5)
            break   
        rospy.sleep(0.1)   
    print hori_pose_Go_stone_set
    print yaw_pitch_normal_set
    print ver_pose_Go_stone_set, gripper_direction_set, gripper_aperture_set
    priority_list = []
    if ver_pose_Go_stone_set == []:
        for index_hori in range(len(hori_pose_Go_stone_set)):
            priority_list.append(('h', index_hori))
    elif hori_pose_Go_stone_set == []:
        for index_ver in range(len(ver_pose_Go_stone_set)):
            priority_list.append(('v', index_ver))
    else:
        for index_hori in range(len(hori_pose_Go_stone_set)):
            for index_ver in range(len(ver_pose_Go_stone_set)):
                if hori_pose_Go_stone_set[index_hori][2]<=ver_pose_Go_stone_set[index_ver][2]-0.002 and ('v', index_ver) not in priority_list:
                    priority_list.append(('v', index_ver))
                v_num = 0
                for element in priority_list:
                    if element[0]=='v':
                        v_num+=1
                if (v_num == len(ver_pose_Go_stone_set) or hori_pose_Go_stone_set[index_hori][2]>ver_pose_Go_stone_set[index_ver][2]-0.002) and ('h', index_hori) not in priority_list:
                    priority_list.append(('h', index_hori))
        for index_ver in range(len(ver_pose_Go_stone_set)):
            if ('v', index_ver) not in priority_list:
                priority_list.append(('v', index_ver))
    print priority_list
    for index in priority_list:
        '''
        if index[0]=='v':
            if ver_pose_Go_stone_set[index[1]]==[None, None, None]:
                continue
            if ver_pose_Go_stone_set[index[1]][2]<0.02:
                continue
            if ver_pose_Go_stone_set[index[1]][2]>0.075:
                continue
            if direct_pick_collision_check(index[1])==False:
                direct_pick_execute(index[1])
                break
        '''
        if index[0]=='h':
            if hori_pose_Go_stone_set[index[1]]==[None, None, None]:
                continue  
            if hori_pose_Go_stone_set[index[1]][2]<0.02:
                continue
            if hori_pose_Go_stone_set[index[1]][2]>0.075:
                continue
            if distane_to_bowl_center(hori_pose_Go_stone_set[index[1]])<0.05:
                scoop4(index[1])
                break
            '''
            else:
                if scoop_collision_check(index[1])==False:
                    scoop_execute(index[1]) 
                    break
            '''

KeyboardInterrupt: 

In [22]:
((0.075-0.07522677506936579)+(0.05-0.050549765559007184)+(0.025-0.024505401570517785)+(0.025-0.02616018528975756))/4.0

-0.00036053187216207854

In [23]:
((0.675-0.6743803927431379)+(0.675-0.6735048336785908)+(0.7-0.6976843453451326)+(0.65-0.6473276838358174))/4.0

0.001775686099330348

In [30]:
Robotiq.goto(robotiq_client, pos=0.021, speed=0.000001, force=1, block=True)
#servo_pub.publish(180)
#softtip()

In [41]:
Robotiq.goto(robotiq_client, pos=0.8, speed=0.000001, force=1, block=True)

In [18]:
tcpTnormal

NameError: name 'tcpTnormal' is not defined

In [9]:
go_to_safe_place()
hardtip()
Robotiq.goto(robotiq_client, pos=0.023, speed=0.1, force=10, block=True)
servo_pub.publish(10)    
pos_x = rob.get_pose().pos.x+0.007
pos_y = rob.get_pose().pos.y-0.001
rospy.sleep(0.5)
rob.translate((0.05-pos_x,0.85-pos_y,0), acc=0.1, vel=0.4, wait=True)
rob.movel_tool((0,0,0,0,0,0),acc=0.1, vel=0.2)
rob.translate_tool((0.0065,0,0),acc=0.1, vel=0.2)

<Transform:
<Orientation: 
array([[-0.01308933,  0.99989511,  0.00619996],
       [ 0.99991303,  0.01307907,  0.00169334],
       [ 0.00161207,  0.00622159, -0.99997935]])>
<Vector: (0.04282, 0.80750, 0.45825)>
>

In [75]:
rob.set_tcp((-0.02, 0, 0.342, 0, 0, 0))
rospy.sleep(0.5)
#if np.cross([fingertip2normal[0], fingertip2normal[1]], [-sin(alpha), cos(alpha)])>0:
rob.movel_tool((0,0,0,0,pi/6,0), acc=0.3, vel=0.8, wait=True)
rob.movel_tool((0,0,0,0,-pi/6,0), acc=0.3, vel=0.8, wait=True)
rob.set_tcp((0, 0, 0, 0, 0, 0))
rospy.sleep(0.5)

In [37]:
        hardtip()
        go_to_safe_place()
        pos_x = rob.get_pose().pos.x+0.007
        pos_y = rob.get_pose().pos.y-0.001

        rob.translate((0.05-pos_x+0.03,0.69-pos_y-0.03,0), acc=0.1, vel=0.4, wait=True)
        rob.translate((0,0,-0.075), acc=0.1, vel=0.4, wait=True)
        rob.translate((0,0.06,0), acc=0.1, vel=0.4, wait=True)
        rob.translate((-0.06,0,0), acc=0.1, vel=0.4, wait=True)
        rob.translate((0,-0.06,0), acc=0.1, vel=0.4, wait=True)
        rob.translate((0.06,0,0), acc=0.1, vel=0.4, wait=True)
        rob.translate((0,0,0.075), acc=0.01, vel=0.05, wait=True)     


<Transform:
<Orientation: 
array([[-0.01329421,  0.99989335,  0.00604565],
       [ 0.99990707,  0.01327565,  0.00310065],
       [ 0.00302006,  0.00608631, -0.99997692]])>
<Vector: (0.07290, 0.66112, 0.45890)>
>

In [20]:
Robotiq.goto(robotiq_client, pos=40*0.14/48.0+1.12/48.0, speed=0.1, force=10, block=True)

In [74]:
go_to_safe_place()
Robotiq.goto(robotiq_client, pos=4*0.14/48.0+1.12/48.0, speed=0.1, force=10, block=True)
rob.movel_tool((0,0,0,0,0,0),acc=0.1, vel=0.2)
tcp2fingertip=m3d.Transform()
tcp2fingertip.pos.x = -0.02 # need change
tcp2fingertip.pos.z = 0.342 # need change
world2tcp = rob.get_pose()
world2tcp.pos.x+=0.004
world2tcp.pos.y+=0.001
world2fingertip=(world2tcp*tcp2fingertip)
rob.translate((0.05-world2fingertip.pos.x,0.85-world2fingertip.pos.y,0), acc=0.3, vel=0.8, wait=True)
#rob.translate((0,0,-0.068), acc=0.1, vel=0.4, wait=True)

<Transform:
<Orientation: 
array([[-0.01299897,  0.99989554,  0.00632014],
       [ 0.99991464,  0.01299036,  0.00140147],
       [ 0.00131922,  0.00633782, -0.99997905]])>
<Vector: (0.04338, 0.86854, 0.45810)>
>

In [5]:
    eeTcam = m3d.Transform()
    eeTcam.pos = (0.076173+0.05, -0.0934057-0.03, 0.0074811)
    eeTcam_e = tf.transformations.euler_from_quaternion([-0.0143125,0.69183,-0.0012,0.722039], axes='sxyz')
    eeTcam.orient.rotate_xb(eeTcam_e[0])
    eeTcam.orient.rotate_yb(eeTcam_e[1])
    eeTcam.orient.rotate_zb(eeTcam_e[2])
    tcpTee = np.array([[0,-1,0,0], [0,0,-1,0], [1,0,0,0], [0,0,0,1]])
    tcp2cam = np.matmul(tcpTee, eeTcam.get_matrix())
    print tcp2cam

[[ 0.02153275 -0.9995875  -0.01900464  0.1234057 ]
 [ 0.99884698  0.02232484 -0.04250071 -0.0074811 ]
 [ 0.04290745 -0.01806757  0.99891567  0.126173  ]
 [ 0.          0.          0.          1.        ]]


In [3]:
softtip()

KeyboardInterrupt: 

In [19]:
rob.set_tcp((0.006, 0, 0.3734, 0, 0, 0))
rospy.sleep(0.5)
rob.movel_tool((0,0,0,0,pi/6,0), acc=0.5, vel=0.8, wait=True)
rob.set_tcp((0, 0, 0, 0, 0, 0))
rospy.sleep(0.5)

In [29]:
go_to_safe_place()

In [31]:
softtip()
hardtip()

In [153]:
    servo_pub.publish(180)
    rospy.sleep(1)


In [116]:
rob.set_tcp((0, 0, 0.35947, 0, 0, 0))
rospy.sleep(1)
print rob.get_pose()
rob.set_tcp((0, 0, 0, 0, 0, 0))
rospy.sleep(1)

<Transform:
<Orientation: 
array([[-0.01308334,  0.99989484,  0.00625631],
       [ 0.99991303,  0.01307268,  0.00174152],
       [ 0.00165955,  0.00627855, -0.99997891]])>
<Vector: (0.04541, 0.54392, 0.09884)>
>


In [24]:
import serial
import signal
import time
serialPort = "/dev/Arduino"
baudRate = 57600
ser = serial.Serial(serialPort, baudRate, timeout=0.5)
time0 = time.time()
def myHandler(signum, frame):
    pass
signal.signal(signal.SIGALRM, myHandler)
signal.setitimer(signal.ITIMER_REAL, 0.001)
ser.write('1180')
signal.setitimer(signal.ITIMER_REAL, 0)
time.sleep(1)
print time.time()-time0
print '2'

1.00144481659
2


In [21]:

softtip()

In [24]:
hori_pose_Go_stone_set[index[1]][0]>point0[0]-0.015 and hori_pose_Go_stone_set[index[1]][0]<point0[0]+0.015 and hori_pose_Go_stone_set[index[1]][1]>point0[1]-0.015 and hori_pose_Go_stone_set[index[1]][1]<point0[1]+0.015 and hori_pose_Go_stone_set[index[1]][2]<point0[2]

True

In [28]:
#go_to_safe_place()
rob.set_tcp((0, 0, 0.36142, 0, 0, 0))
rospy.sleep(0.5)

move = m3d.Transform((0.8-rob.get_pose().pos.y+0.001, 0.05-rob.get_pose().pos.x-0.004, 0, 0, 0, 0))    
rob.add_pose_tool( move, acc=0.3, vel=1, wait=True, command="movel", threshold=None)
rob.set_tcp((0, 0, 0, 0, 0, 0))
rospy.sleep(0.5)

In [39]:
rob.set_tcp((0, 0, 0.36142, 0, 0, 0))
rospy.sleep(0.5)
rob.get_pose()

<Transform:
<Orientation: 
array([[-0.01297587,  0.99989633,  0.00624184],
       [ 0.99991459,  0.01296587,  0.00163983],
       [ 0.00155873,  0.00626258, -0.99997917]])>
<Vector: (0.04543, 0.54389, 0.09685)>
>

In [23]:
go_to_end_place()

In [17]:
      Robotiq.goto(robotiq_client, pos=0.8, speed=0.5, force=10, block=True)


In [20]:
softtip()

In [121]:
    Hong_joint0 = radians(81.54)
    Hong_joint1 = radians(-94.03)
    Hong_joint2 = radians(-105.84)
    Hong_joint3 = radians(-74.76)
    Hong_joint4 = radians(118.75)
    Hong_joint5 = radians(-189.52)
    rob.movej((Hong_joint0,Hong_joint1, Hong_joint2, Hong_joint3, Hong_joint4, Hong_joint5), 0.6, 1.5)

[1.2988009452819824,
 -1.766688648854391,
 -1.8735087553607386,
 -1.3008382956134241,
 2.2578067779541016,
 -3.4949784914599817]

In [173]:
go_to_safe_place()
Robotiq.goto(robotiq_client, pos=2.5*0.14/48.0+1.12/48.0, speed=0.5, force=10, block=True)  
signal.signal(signal.SIGALRM, myHandler)
signal.setitimer(signal.ITIMER_REAL, 0.005)
ser.write('1030')
signal.setitimer(signal.ITIMER_REAL, 0)
time.sleep(1)
softtip()

KeyboardInterrupt: 

In [30]:
go_to_safe_place()
rob.translate((0,0,-0.035), acc=0.1, vel=0.1, wait=True)

<Transform:
<Orientation: 
array([[-0.01306666,  0.99989477,  0.00630122],
       [ 0.99991333,  0.01305622,  0.00169527],
       [ 0.00161282,  0.00632282, -0.99997871]])>
<Vector: (0.04322, 0.54326, 0.42337)>
>

In [17]:
switching_stone_coordinate_set

[[0.04748682930698754, 0.6772187228828948, 0.03650951687510667],
 [0.04285301004949358, 0.6773116203834977, 0.04474243949710238],
 [0.020558080749336436, 0.7007856115591724, 0.05011494897992741],
 [0.04654879802776229, 0.7035219271723925, 0.06111810419601599],
 [0.03623374456712877, 0.6653812126120116, 0.05419025420948426],
 [0.06196829769648066, 0.687646955878628, 0.053823396522124034],
 [0.05775511489062693, 0.6660403423087387, 0.06140280538363507],
 [0.06572986499869145, 0.7087816075687934, 0.04626566310967978],
 [0.0769745887897275, 0.6758826107559193, 0.046508792057820714],
 [0.022329528772110585, 0.6842764932194296, 0.04333823667183745],
 [0.04992898805327124, 0.7293906219163122, 0.03750523331270783],
 [0.023780934590662446, 0.7260079873804601, 0.03652088731031233],
 [0.005551637499277247, 0.6835217107176383, 0.03491895253859206],
 [0.01950094174002195, 0.660207221543091, 0.036879290367629936],
 [0.03986341513255099, 0.6496814636328898, 0.036299742084113296],
 [0.0739991948029581

In [30]:
signal.setitimer(signal.ITIMER_REAL, 0.001)
ser.write('1180')
signal.setitimer(signal.ITIMER_REAL, 0)
time.sleep(1)


In [21]:
Robotiq.goto(robotiq_client, pos=40*0.14/48.0+1.12/48.0, speed=0.1, force=10, block=True)

In [30]:
Robotiq.goto(robotiq_client, pos=0.047, speed=0.5, force=10, block=True)

In [22]:
signal.setitimer(signal.ITIMER_REAL, 0.001)
ser.write('1180')
signal.setitimer(signal.ITIMER_REAL, 0)
time.sleep(1)

In [52]:
go_to_safe_place()

In [51]:
for i in range(28):
    print i
    skim_grasping()

0


KeyboardInterrupt: 

In [46]:
(0.047-1.12/48.0)*48/0.14

8.114285714285712

In [32]:
go_to_home()

In [53]:
Robotiq.goto(robotiq_client, pos=40*0.14/48.0+1.12/48.0, speed=0.5, force=10, block=True)  

Exception in thread Thread-75:
Traceback (most recent call last):
  File "/home/terry/anaconda3/envs/ros_control/lib/python2.7/threading.py", line 801, in __bootstrap_inner
    self.run()
  File "/home/terry/anaconda3/envs/ros_control/lib/python2.7/site-packages/urx/ursecmon.py", line 291, in run
    data = self._get_data()
  File "/home/terry/anaconda3/envs/ros_control/lib/python2.7/site-packages/urx/ursecmon.py", line 338, in _get_data
    tmp = self._s_secondary.recv(1024)
timeout: timed out

Exception in thread Thread-11:
Traceback (most recent call last):
  File "/home/terry/anaconda3/envs/ros_control/lib/python2.7/threading.py", line 801, in __bootstrap_inner
    self.run()
  File "/home/terry/anaconda3/envs/ros_control/lib/python2.7/site-packages/urx/ursecmon.py", line 291, in run
    data = self._get_data()
  File "/home/terry/anaconda3/envs/ros_control/lib/python2.7/site-packages/urx/ursecmon.py", line 338, in _get_data
    tmp = self._s_secondary.recv(1024)
timeout: timed out